In [1]:
"""
this block is needed at the beginning of all notebooks to be able to load modules
"""
import os
import sys
from pathlib import Path
import pandas as pd
import math
import numpy as np
from scipy.signal import detrend
import pywt

# Automatically find and append the project root
root_path = Path.cwd().resolve().parents[1]  # adjust level as needed
data_path = os.path.join(root_path, "data")
src_path = os.path.join(root_path, "src")

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))


In [2]:
print(src_path)

E:\vatecas\src


In [3]:
from signal_modules.base.signal_module import SignalModule
from regression.Kalman_filter import KalmanFilter

In [4]:
noise = 1e-3
kalman = KalmanFilter(process_noise=noise, obs_noise_scale=3 * noise)

In [5]:
signal_module = SignalModule(oracle=kalman, smoothing_window=1)

In [6]:
initial_time = pd.to_datetime('2023-01-01T12:00:00+00:00')
final_time = pd.Timestamp('2024-01-01T12:00:00+00:00')

tickers = ["SPY", "TSLA", "NVDA"]  # List of tickers
data_files = [os.path.join(data_path, f"{ticker}.parquet") for ticker in tickers]
data = {
    ticker: pd.read_parquet(path).loc[initial_time:final_time]
    for ticker, path in zip(tickers, data_files)
}
timeline = pd.DatetimeIndex(
    pd.to_datetime(sorted(set().union(*[df.index for df in data.values()])))
)
data = {tk: df.reindex(timeline).ffill() for tk, df in data.items()}
closing_data = {tk: df['adjClose'] for tk, df in data.items()}

In [7]:
signal_module.generate_signals(closing_data, past_predictions={})

SignalResult(signals=array([-0.2308353 , -0.96402758,  0.05093989]), raw_predictions=array([468.11346469, 252.47432698,  49.40193131]), smoothed_predictions=array([[468.28610956, 255.54033994,  49.32377249],
       [468.11346469, 252.47432698,  49.40193131]]), volatility_adjustments=array([0.20873269, 0.1       , 0.1       ]), tickers=['SPY', 'TSLA', 'NVDA'])

In [8]:
tomorrow = kalman.predict(closing_data)

In [9]:
pd.DataFrame(closing_data).tail()

,SPY,TSLA,NVDA
2023-12-22 00:00:00+00:00,466.254466,252.54,48.812010
2023-12-26 00:00:00+00:00,468.223238,256.61,49.260844
2023-12-27 00:00:00+00:00,469.069810,261.44,49.398793
2023-12-28 00:00:00+00:00,469.247000,253.18,49.503755
2023-12-29 00:00:00+00:00,467.888547,248.48,49.503755
